In [4]:
import pandas as pd
import numpy as np

In [5]:
df = pd.DataFrame({'values': np.random.uniform(-10, 10, 10)})
print(df)

     values
0  0.773143
1 -8.716816
2 -2.222944
3  1.913245
4  2.269296
5 -8.125693
6 -8.375250
7 -7.668111
8  9.331868
9  4.460050


In [6]:
def float_to_words(num: float) -> str:
    d = "zero one two three four five six seven eight nine".split()
    s = str(num)
    return " ".join((["minus"] if s.startswith("-") else (["plus"] if s.startswith("+") else [])) +
                    ["point" if c=="." else d[int(c)] for c in s.lstrip("+-") if c.isdigit() or c=="."])

# Examples
print(float_to_words(-4.61))   # "minus four point six one"
print(float_to_words(+7.25))   # "plus seven point two five"
print(float_to_words(9.0))     # "nine point zero"

minus four point six one
seven point two five
nine point zero


In [7]:
def words_to_float(text: str) -> float:
    num_map = dict(zip(
        ["zero","one","two","three","four","five","six","seven","eight","nine"],
        "0123456789"
    ))
    int_part, _, frac_part = text.lower().partition("point")
    return float(
        "".join(num_map.get(w, "") for w in int_part.split()) +
        ("." + "".join(num_map.get(w, "") for w in frac_part.split()) if frac_part else "")
    )

# Examples
print(words_to_float("four point six one zero"))  # 4.610
print(words_to_float("seven point two five"))     # 7.25
print(words_to_float("nine"))                     # 9.0

4.61
7.25
9.0


In [8]:
def words_to_float(text: str) -> float:
    num_map = {w: str(i) for i, w in enumerate(
              ["zero","one","two","three","four","five","six","seven","eight","nine"]
    )}
    int_part, _, frac_part = text.lower().partition("point")
    return float(
        "".join(num_map.get(w, "") for w in int_part.split()) +
        ("." + "".join(num_map.get(w, "") for w in frac_part.split()) if frac_part else "")
    )

# Examples
print(words_to_float("four point six one zero"))  # 4.610
print(words_to_float("seven point two five"))     # 7.25
print(words_to_float("nine"))                     # 9.0

4.61
7.25
9.0


In [ ]:
def from_or_to_number(value: str | float) -> float | str:
    if value in [None, ""]:
        return None

    num_map = dict()
    if type(value) == str:
        num_map = { w:str(i) for i, w in enumerate(
                      ["zero", "one", "two", "three", "four", "five", "six", "seven", "eight", "nine"]
        )}
        i, _, f = value.lower().partition("point")
        return float("".join([ "minus" ] if i.startswith("-") else [ "plus" ] if i.startswith("+") else [] +
                             [ "".join(num_map.get(w, "") for w in i.split()) ] +
                             [ "." ] +
                             [ "".join(num_map.get(w, "") for w in f.split()) ]
                            )
        )
    elif type(value) in [ int, float ]:
        num_map = { str(i):w for i, w in enumerate(
                      ["zero", "one", "two", "three", "four", "five", "six", "seven", "eight", "nine"]
        )}
        s = str(value)
        return " ".join([ "minus"] if s.startswith("-") else ["plus"] if s.startswith("+") else [] +
                        [ "point" if c=="." else num_map[c] for c in s if c.isdigit()]
        )
    else:
        return None



# Examples
print(from_or_to_number("four point six one zero"))  # 4.610
print(from_or_to_number("seven point two five"))     # 7.25
print(from_or_to_number("nine"))                     # 9.0

print(from_or_to_number(4.610))                      # "four point six one zero"
print(from_or_to_number(7.25))                       # "seven point two five"
print(from_or_to_number(9.0))                        # "nine"